In [1]:
import pandas as pd
from constants import DATA_PATH, EOS_FILE, SENTINEL_FILE

sentinel = pd.read_csv(DATA_PATH / SENTINEL_FILE)
eos = pd.read_csv(DATA_PATH / EOS_FILE)

In [2]:
from constants import X_cols_eos, X_cols_sentinel, y_col

X_sentinel = sentinel[X_cols_sentinel].values
y_sentinel = sentinel[y_col].values

X_eos = eos[X_cols_eos].values
y_eos = eos[y_col].values

In [4]:
from model_experiments import ClassificationExperiment
from constants import class_labels

ce = ClassificationExperiment(X_eos, y_eos, satellite_name='EOS-04', labels=class_labels, print_stats=True, 
                                split_type='train-test', train_size=0.8, test_size=0.2)

eos_results = ce.run_experiment()


--- Experiment Data Initialized ---
Total Samples: 2528
Split Type:    'train-test'
  - Train:   80.0% (2022 samples)
  - Test:    20.0% (506 samples)
Total in splits: 2528
-----------------------------------


--- Running Model: RF ---
Test Acc - 36.1660%

--- Running Model: XGB ---
Test Acc - 35.1779%

--- Running Model: ADA ---
Test Acc - 39.1304%

--- Running Model: SVC ---
Test Acc - 38.1423%
Metrics saved successfully.


In [5]:
ce = ClassificationExperiment(X_sentinel, y_sentinel, satellite_name='Sentinel-1', labels=class_labels, print_stats=True, 
                                split_type='train-test', train_size=0.8, test_size=0.2)

sentinel_results = ce.run_experiment()


--- Experiment Data Initialized ---
Total Samples: 1816
Split Type:    'train-test'
  - Train:   80.0% (1452 samples)
  - Test:    20.0% (364 samples)
Total in splits: 1816
-----------------------------------


--- Running Model: RF ---
Test Acc - 38.1868%

--- Running Model: XGB ---
Test Acc - 39.0110%

--- Running Model: ADA ---
Test Acc - 42.5824%

--- Running Model: SVC ---
Test Acc - 41.4835%
Metrics saved successfully.
